In [1]:
import numpy as np
import qfunk.utility as qut
import qfunk.generator as qg

# Basic quantum information utility functions

This notebook will concern the basic manipulation of quantum information objects. This is not a complete description of the package but covers some of the more commonly desired functions.

## Contents
1. Random generation
2. Partial trace
3. Partial transpose and system permute



## 1. Random generation

Qfunk supports the random generation of a number of useful objects. At the most basic level, this includes random density operators over the [Haar measure](https://pennylane.ai/qml/demos/tutorial_haar_measure.html) from the generator module.

In [2]:
random_density = qg.rand_rho(n=4)

Which is indeed a square trace one positive semidefinite matrix

In [3]:
np.shape(random_density)

(4, 4)

In [4]:
np.trace(random_density)

(0.9999999999999999+5.990217002982412e-18j)

In [5]:
np.linalg.eigvals(random_density)

array([0.27751227+3.52866420e-18j, 0.28381057+3.11956072e-17j,
       0.22574785-2.65116394e-20j, 0.21292931-1.88413030e-17j])

which is hermitian and can be checked using the dagger utility function:

In [6]:
np.isclose(qut.dagger(random_density), random_density, atol=1e-15)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

One can also generate random pure states by setting the **pure** keyword to True.

In [7]:
random_density = qg.rand_rho(4, pure=True)
max(abs(np.linalg.eigvals(random_density)))

1.0000000000000004

In addition to random state generation, random unitaries of arbitrary size are also easily generated according to the Haar measure. 

In [8]:
U = qg.random_unitary(n=4)
# check that UU^\dagger is the identity (same for U^\dagger U )
np.isclose(U @ qut.dagger(U), np.eye(4), atol=1e-15)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

## 2. Partial trace

The partial trace of a bipartite state $\rho_{AB}$ with respect to the subsystem $A(B)$ is computed by calling the trans `trace_x` function and specifying the index (beginning at $0$) of the corresponding Hilbert space. Computing the operation

\begin{equation}
\rho_A = tr_B [\rho_{AB}] .
\end{equation}

First we generate a maximally entangled bipartite state using `ent_gen` from the generator module as an example and specify the two component subsystems to be three dimensional vector spaces.

In [9]:
rho_AB = qg.ent_gen(n=3)

Then we trace out the second subsystem by indicating its system index `[1]` and the dimension of the underlying Hilbert space in an ordered list `[3,3]`

In [10]:
rho_A = qut.trace_x(rho_AB, sys=[1], dim=[3,3])

Which outputs $\rho_A$, the reduced state on subsystem $A$.

In [11]:
rho_A

array([[0.33333333+0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.33333333+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.33333333+0.j]])

which is of course the maximally mixed state on three dimensions. We can get a bit more sophisticated and trace out multiple systems of different dimension using the `tn_product` function, which computes the tensor product of its arguments:

In [15]:
# generate random states of differing dimension for the subsystem
rho_A = qg.rand_rho(n=2, pure=True)
rho_B = qg.rand_rho(n=3, pure=True)
rho_C = qg.rand_rho(n=4, pure=True)
# create a list object for easy passing to the tensor product function, can also just use qut.tn_product(rho_A, rho_B, rho_C)
subsystems = [rho_A, rho_B, rho_C]
# compute the tensor product state
rho_ABC = qut.tn_product(*subsystems)

This function is also useful for computing tensor product powers quickly like $\rho_A^{\otimes3}$, by passing

In [13]:
alist = [rho_A]*3
rho_AAA = qut.tn_product(*alist)

To recover the reduced states, we specify the partial trace as before, specifying the indices of subsystems $B,C$ and the ordered list of subsystem dimensions. 

In [14]:
rho_A_reduced = qut.trace_x(rho_ABC, [1,2],[2,3,4])
# check if reduced state is still still rho_A
np.isclose(rho_A, rho_A_reduced, atol=1e-15)

array([[ True,  True],
       [ True,  True]])

## 3. Partial transpose and subsystem permute

The partial transpose `trans_x` and subsystem functions `sys_permute` operations act in a near identical way as the partial transpose. For the partial tranpose only the operator, the subsystems to permute indexed from zero and the subsystem dimensions need to be specified:

In [20]:
# construct the state
rho_A = qg.rand_rho(n=2)
rho_B = qg.rand_rho(n=3)
rho_AB = qut.tn_product(rho_A, rho_B)
# compute partial transpose
rho_AtB = qut.trans_x(rho_AB, sys=[0], dim=[2,3])
# compare to ground truth
np.isclose(rho_AtB, qut.tn_product(np.transpose(rho_A), rho_B), atol=1e-15)

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

The subsytem permute works in a similar way except all susbsytems and their new postions must be specified

In [27]:
# construct the state
rho_A = qg.rand_rho(n=2)
rho_B = qg.rand_rho(n=2)
rho_C = qg.rand_rho(n=2)
rho_ABC = qut.tn_product(rho_A, rho_B, rho_C)
# swap ABC for BCA i.e. [0,1,2] -> [1,2,0]
rho_BCA = qut.sys_permute(rho_ABC, perm=[1,2,0], dim=[2,2,2])

Critically, the dimension list is the ordered list of dimensions for the \textit{input} state, not the desired output state.

In [28]:
np.isclose(rho_BCA, qut.tn_product(rho_B, rho_C, rho_A), atol=1e-15)

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])